# 爬取各隊各年的薪資

####  這個R程式會去爬取[網站資料](https://www.spotrac.com/mlb/payroll/年度)，根据年份匯出csv 檔


### 匯入函數庫
* xml2 : rvest 所需函式庫
* rvest: 用來爬蟲
* dplyr: 清理DataFrame
* readr: 資料處理(parse_number eg.turn "$100" into 100)

In [1]:
library(xml2)
library(rvest)
library(dplyr)
library(readr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



### 定義R函式，此函式爬網站，並組成data.frame，轉換特定column 的型態
define a function : scrapping each year payrol
given url vectors ,the function will scrape each url and it's node ,and return the data.frame 

In [22]:
crawler_each_year <- function(url,node,colname_node){
    rd <- read_html(url)
    node_name <- html_nodes(rd,node)
    text <- html_text(node_name)
    
    #  turn text vector into frame data.frame
    tmp_row <- character()
    frame <- data.frame(matrix(ncol = 8),stringsAsFactors=FALSE)
    col_num <- 0
    for (i in 1 : length(text)){
        if(i %% 8 != 0 ){
            tmp_row <- c(tmp_row,text[i])
        }
        else{
           tmp_row <- c(tmp_row,text[i])
           frame <- rbind(frame,tmp_row)
           tmp_row <- character() 
        }
        col_num <- 1
    }
    # remove the row with NA value
    frame <- frame[complete.cases(frame),]
    
    # turn the character colnumn into nummeric column
    for (i in 1:ncol(frame)){
        tmp_col <- frame[i]
        if(i != 1){
            tmp_col <- sapply(frame[i],parse_number)
        }
        
        if(i == 1){
            new_frame <- tmp_col
        }
        else {
            new_frame <- cbind(new_frame,tmp_col)
        }
        
    }
    
    # frame is the data.frame without the colnames,colnames need to be added
    node_colname <- html_nodes(rd,colname_node)
    colname <- html_text(node_colname)
    if(length(colname) != 8){
        stop("colname is wrong !!")
    }
    colnames(new_frame) <- colname
    # rank the Team
    new_frame <- new_frame %>% mutate(Rank = row_number())
    return(new_frame)
}

x <- crawler_each_year("https://www.spotrac.com/mlb/payroll/2011/",".result , td.xs-hide , .noborderleft","#payroll , #susp , #buried , #dead , #ir , #active , #players , #team")
head(x,20)                                                                                            

Team,Roster,25-Man Payroll,Injured Reserve,Retained,Buried,Suspended,2011 Total Payroll,Rank
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
New York Yankees,41,212609678,0,950000,0e+00,0,213559678,1
Boston Red Sox,47,164517590,0,8239278,0e+00,0,172553261,2
Philadelphia Phillies,38,169799882,0,168300,0e+00,0,169968182,3
Los Angeles Angels of Anaheim,38,133613959,0,13034600,0e+00,0,146648559,4
New York Mets,38,96163418,0,45668423,0e+00,0,141831841,5
Chicago Cubs,40,124652698,0,14825000,0e+00,0,139477698,6
Chicago White Sox,40,123584951,0,7038798,0e+00,0,130623749,7
San Francisco Giants,39,124882250,0,1100000,0e+00,0,125982250,8
Los Angeles Dodgers,41,98679615,0,19989000,0e+00,0,118668615,9


### 爬不同的網站，整理其url 

In [20]:
url_vect <- rep("https://www.spotrac.com/mlb/payroll/20",8)
tail_num <- 11:18
url_vect <- paste(url_vect,tail_num,sep = "") %>% paste("/",sep = "")

### 不同年份的DataFrame整理成list 內的元素

In [ ]:
each_year_payrol <- list()
tmp_frame <- data.frame()
for(i in 1:length(url_vect)){
    tmp_frame <- crawler_each_year(url_vect[i],".result , td.xs-hide , .noborderleft","#payroll , #susp , #buried , #dead , #ir , #active , #players , #team")
    #each_year_payrol <- c(each_year_payrol,tmp_frame)  
    # list append data.frame is wrong,it's turn each row in dataframe into vector then append to list 
    # solution :
    each_year_payrol <- append(each_year_payrol,list(tmp_frame))  
    tmp_frame <- data.frame()
}
x <- each_year_payrol

### 更改工作路徑

In [21]:
setwd("D:\\github 工作區\\MLB-baseball\\各年各隊薪資")

### 匯出各年份資料為csv檔

In [19]:
# ouptut with excel form every year
for (i in 1 : length(each_year_payrol)){
    year <- as.character(2010 + i)
    file_name <- paste(year,".csv",sep="")
    write.table(each_year_payrol[i],file_name,sep=",",row.names = FALSE)
}